# IE 306  SYSTEM SIMULATION
## FURKAN KESKIN - 2018400150
## SINEM KOCOGLU - 2020400339

In [240]:
import numpy as np
from queue import Queue, PriorityQueue

SEED = 2018400150 + 2020400339

nurses = 4  # S
beds = 6    # K
p_1 = 0.25
lambd = 1
mu_T = 0.3125
mu_s = 0.16
mu_cb = 0.1666666667

## PART 1.2

In [241]:
iterations = 1500

def Generate_Interarrival():
    return np.random.exponential(1/lambd, iterations)

def Generate_Nurse_Service_Time():
    return np.random.exponential(1/mu_T, iterations)

def Generate_Hospital_Healing_Time():
    return np.random.exponential(1/mu_cb, iterations)

def Generate_Home_Healing_Time(condition):
    if condition == 's':
        return np.random.exponential(1/mu_s, iterations)
    elif condition == 'c':
        alfa = np.random.uniform(1.25, 1.75)
        return np.random.exponential(alfa/mu_cb, iterations)
    
def Generate_Conditions():
    condition_probs = np.random.uniform(0, 1, iterations)
    return np.array(['s' if i <= p_1 else 'c' for i in condition_probs])

In [242]:
np.random.seed(SEED)

interarrivals = Generate_Interarrival()
nurse_service_times = Generate_Nurse_Service_Time()
hospital_healing_times = Generate_Hospital_Healing_Time()
home_healing_times_stable = Generate_Home_Healing_Time('s')
home_healing_times_critical = Generate_Home_Healing_Time('c')
arrivals = np.roll(interarrivals.cumsum(), 2)
conditions = Generate_Conditions()

In [243]:
def Arrival():
    global busy_nurses, nurse_idx
    
    # adding next arrival to the FEL
    FEL.put((arrivals[pid-already_in_hospital+1], 'A', pid+1))
    
    if busy_nurses < nurses:
        FEL.put((simulation_time + nurse_service_times[nurse_idx], 'DN', pid))
        busy_nurses += 1
        nurse_idx += 1
    else:
        WQ.put((simulation_time, pid))
            
    

def Departure_Triage():
    global busy_nurses, occupied_beds, nurse_idx, condition_idx, hospital_idx, home_stable_idx, home_critical_idx
    global people_rejected, total_nurse_triage_time
    
    # updating total nurse triage time
    total_nurse_triage_time += nurse_service_times[pid-initially_occupied_beds]
    
    # should be independent from the SEED
    # condition = 's' if np.random.uniform(0, 1) <= p_1 else 'c'
    condition = conditions[condition_idx]
    condition_idx += 1
    
    # departure of the current patient from the nurse triage
    if condition == 's':
        healing_time_dict[pid] = home_healing_times_stable[home_stable_idx]
        
        FEL.put((simulation_time + home_healing_times_stable[home_stable_idx], 'H', pid))
        home_stable_idx += 1
    elif condition == 'c':
        if occupied_beds < beds:
            healing_time_dict[pid] = hospital_healing_times[hospital_idx]
            
            FEL.put((simulation_time + hospital_healing_times[hospital_idx], 'DB', pid))
            occupied_beds += 1
            hospital_idx += 1
        else:
            healing_time_dict[pid] = home_healing_times_critical[home_critical_idx]
            
            FEL.put((simulation_time + home_healing_times_critical[home_critical_idx], 'H', pid))
            home_critical_idx += 1
            people_rejected += 1
            
    # arrangements for the FEL
    if WQ.empty():
        busy_nurses -= 1
        
    else:
        # arrival time queue'da ne kadar beklediğiyle alakalı
        arrival_time_of_incoming, pid_of_incoming = WQ.get()
        FEL.put((simulation_time + nurse_service_times[nurse_idx], 'DN', pid_of_incoming))
        nurse_idx += 1
        
        

def Treated_at_Hospital():
    global occupied_beds, healed_patients
    global total_healing_time 
    
    total_healing_time += healing_time_dict[pid]
     
    occupied_beds -= 1
    healed_patients += 1
    

def Healed_at_Home():
    global healed_patients, healed_at_home
    global total_healing_time 
    
    total_healing_time += healing_time_dict[pid]
    
    healed_at_home += 1
    healed_patients += 1
    
    
def Advance_Time():
    global simulation_time, event_type, pid, event_no
    simulation_time, event_type, pid = FEL.get()
    event_no += 1
    
    
def Execute_Event():
    global prev_simulation_time, empty_nurses_time, empty_beds_time, empty_nurses_and_beds_time, total_occupied_beds
    
    if busy_nurses != nurses:
        empty_nurses_time += (simulation_time - prev_simulation_time)
        
    if occupied_beds != beds:
        empty_beds_time += (simulation_time - prev_simulation_time)
        
    if busy_nurses != nurses and occupied_beds != beds:
        empty_nurses_and_beds_time += (simulation_time - prev_simulation_time)
    
    
    if event_type == 'A':
        Arrival()
    elif event_type == 'DN':
        Departure_Triage()
    elif event_type == 'DB':
        Treated_at_Hospital()
    elif event_type == 'H':
        Healed_at_Home()
        
           
    prev_simulation_time = simulation_time
        
    total_occupied_beds += occupied_beds
        
    print(event_no, round(simulation_time, 3), event_type, pid, hospital_idx, FEL.queue[0], FEL.qsize(), WQ.qsize(), busy_nurses, occupied_beds, healed_patients)
        
    Advance_Time()
 

## PART 2.2

In [244]:
 for starting_condition in ((0, 0), (nurses//2, beds//2), (nurses, beds)):
    for termination_limit in ((20),):
        FEL = PriorityQueue()              # Future Event List  
        WQ = Queue()                       # Waiting Queue for Nurse Service
        healing_time_dict = {}             # pid: hospital_healing_times[hospital_idx]

        # initial set-up
        prev_simulation_time = 0
        simulation_time = 0
        event_type = 'A'
        pid = 1
        event_no = 1
        
        # global constants 
        initially_busy_nurses = starting_condition[0]
        initially_occupied_beds = starting_condition[1]
        already_in_hospital = initially_busy_nurses + initially_occupied_beds

        # global variables
        busy_nurses = starting_condition[0]   
        occupied_beds = starting_condition[1]

        # initial activity indices 
        nurse_idx = 0
        condition_idx = 0
        hospital_idx = 0
        home_stable_idx = 0
        home_critical_idx = 0

        healed_patients = 0

        # 0, 1, 2 .... idx-1

        # System starts with a patient with ID = 1 in simulation time = 0
        # t, event, ID
        
        # creating initial patients in nurse triage
        for _ in range(busy_nurses):
            FEL.put((nurse_service_times[nurse_idx], 'DN', pid))
            nurse_idx += 1
            pid += 1
            
        # creating initial patients in hospital beds
        for _ in range(occupied_beds):
            healing_time_dict[pid] = hospital_healing_times[hospital_idx]
            
            FEL.put((hospital_healing_times[hospital_idx], 'DB', pid))
            hospital_idx += 1
            pid += 1
            
        # statistics
        empty_nurses_time = 0
        empty_beds_time = 0
        empty_nurses_and_beds_time = 0
        people_rejected = 0
        total_nurse_triage_time = 0
        total_occupied_beds = 0
        healed_at_home = 0
        total_healing_time = 0

        while healed_patients < termination_limit:
            Execute_Event()
            
        
        print()
        print(simulation_time, empty_nurses_time, empty_beds_time, empty_nurses_and_beds_time)
        print(pid, people_rejected / pid)
        print(total_nurse_triage_time, nurses*simulation_time)
        print(total_occupied_beds, beds*event_no)
        print(healed_at_home, pid)
        print(total_healing_time, healed_patients)
        
        print("-----------------------------------------------------------------")



1 0 A 1 0 (0.2410704873911317, 'A', 2) 2 0 1 0 0
2 0.241 A 2 0 (0.8884782567780792, 'A', 3) 3 0 2 0 0
3 0.888 A 3 0 (2.441923752341665, 'DN', 1) 4 0 3 0 0
4 2.442 DN 1 1 (3.419874382030668, 'DN', 3) 4 0 2 1 0
5 3.42 DN 3 2 (4.110111901162006, 'DN', 2) 4 0 1 2 0
6 4.11 DN 2 3 (4.738869535989139, 'DB', 2) 4 0 0 3 0
7 4.739 DB 2 3 (5.392597195791888, 'A', 4) 3 0 0 2 1
8 5.393 A 4 3 (6.278577000540807, 'DN', 4) 4 0 1 2 1
9 6.279 DN 4 4 (6.466939768343855, 'A', 5) 4 0 0 3 1
10 6.467 A 5 4 (9.000690331158372, 'A', 6) 5 0 1 3 1
11 9.001 A 6 4 (9.319872369193263, 'A', 7) 6 0 2 3 1
12 9.32 A 7 4 (10.322908724059928, 'DN', 7) 7 0 3 3 1
13 10.323 DN 7 5 (10.421718094088671, 'A', 8) 7 0 2 4 1
14 10.422 A 8 5 (10.428811150541199, 'DN', 8) 8 0 3 4 1
15 10.429 DN 8 6 (10.58367661205897, 'DB', 8) 8 0 2 5 1
16 10.584 DB 8 6 (11.00700756778249, 'A', 9) 7 0 2 4 2
17 11.007 A 9 6 (11.235087841542711, 'A', 10) 8 0 3 4 2
18 11.235 A 10 6 (11.317734044043737, 'A', 11) 9 0 4 4 2
19 11.318 A 11 6 (11.837949332